# Training of College helper Chatbot
### Abeera Malakar

In [28]:
import random
import json
import pickle
import numpy as np

In [29]:
import nltk
from nltk.stem import WordNetLemmatizer

In [30]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.optimizers.legacy import SGD

In [31]:
# load the intents

with open('intents.json','r') as f:
   intents= json.load(f)

Now we're goint to lemmatize each word
=> works,worked,working==== work

In [32]:
lt=WordNetLemmatizer()

In [33]:
words=[]
classes=[]
documents=[]
ignore_letters=['?','!','.',',']

In [34]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list=nltk.word_tokenize(pattern)#tokenize=> "helo me god"==["hello","me","god"]
        words.extend(word_list) #appending each word in word list
        documents.append((word_list,intent['tag'])) # storing word and their tags
        if intent['tag'] not in classes:
            classes.append(intent['tag'])#appending the tags in classes
#now lemmatizing the words        
words=[lt.lemmatize(word) for word in words if word not in ignore_letters]
words=sorted(set(words))
classes=sorted(set(classes))

In [35]:
print(words)

["'s", '(', ')', 'AC', 'AI/ML', 'AI/Ml', 'About', 'Ac', 'Are', 'Bye', 'Cafetaria', 'Can', 'Chemical', 'Civil', 'College', 'Computer', 'Contact', 'Do', 'Does', 'Engineering', 'Events', 'Food', 'Give', 'Good', 'Goodbye', 'HOD', 'Have', 'Hello', 'Hi', 'Holiday', 'How', 'I', 'IT', 'Information', 'Is', 'Leaving', 'Library', 'ME', 'Mechanical', 'See', 'Semester', 'Sports', 'Syllabus', 'Technology', 'Tell', 'Thank', 'Thanks', 'UNI', 'What', 'Whats', 'When', 'Where', 'Which', 'Who', 'Will', 'You', 'a', 'about', 'active', 'activity', 'address', 'admision', 'admission', 'against', 'allotment', 'am', 'an', 'and', 'antiragging', 'any', 'anyone', 'are', 'as', 'asshole', 'at', 'attend', 'automobile', 'available', 'average', 'be', 'between', 'big', 'bitch', 'book', 'boy', 'branch', 'bring', 'building', 'by', 'bye', 'cafetaria', 'call', 'called', 'campus', 'can', 'canteen', 'capacity', 'case', 'casuals', 'ce', 'chatting', 'chemical', 'civil', 'code', 'college', 'come', 'committee', 'comp', 'company', 

In [36]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

now we will use bag of words technique to convert the words to neumerical values as neural network uses neumerical values

In [37]:
training=[]
output_empty=[0]*len(classes)

for document in documents:
    bag=[] #crearing an empty bag
    word_patterns=document[0]; #as the word is the first element of document
    word_patterns=[lt.lemmatize(word.lower()) for word in word_patterns] #lemmatizing the lowercase words for training list
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
        
    output_row=list(output_empty)
    output_row[classes.index(document[1])]=1 #mark the selected class of document as 1
    training.append([bag,output_row])

In [38]:
#shuffel the data randomly for training

random.shuffle(training)
training=np.array(training)

C:\Users\Sleo\AppData\Local\Temp\ipykernel_17232\1965495709.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training=np.array(training)


In [39]:
#input & output data

train_x=list(training[:,0])
train_y=list(training[:,1])

##### Model Building

In [40]:
model=Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))

sgd=SGD(learning_rate=0.01,decay=1e-6,momentum=0.9,nesterov=True)

In [41]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               39168     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 38)                2470      
                                                                 
Total params: 49,894
Trainable params: 49,894
Non-trainable params: 0
_________________________________________________________________


In [42]:
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

In [ ]:
hist=model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=10,verbose=1)

In [ ]:
model.save('CollegeBuddy_model.h5',hist)